In [66]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import pandas as pd
import time
import random
import datetime
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 8]
pd.set_option("display.precision", 3)

In [67]:
opp_stats = pd.read_csv('opponent_stats',index_col=0)
schedule = pd.read_csv('schedule',index_col=0)
data = pd.read_csv('data.csv')
data['Date'] = pd.to_datetime(data['Date'])
lines = pd.read_csv('betting_lines_1',index_col=0)
lines = lines[lines.stat != 'Fantasy Score']
lines2 =pd.read_csv('betting_lines_2',index_col=0)
lines2 = lines2[lines2.stat != 'Fantasy Score']
lines3 =pd.read_csv('betting_lines_3',index_col=0)
lines3 = lines3[lines3.stat != 'Fantasy Score']

In [68]:
lines['player_name'] = lines['player_name'].replace(['Nicolas Claxton'], 'Nic Claxton')
lines['player_name'] = lines['player_name'].replace(['Lebron James\t'], 'Lebron James')
lines['Date'] = '2023-02-15'
lines
lines2['Date'] = '2023-02-16'
lines2
lines3['Date'] = '2023-02-23'
lines3

,player_name,team,line,stat,id,Date
0,Jayson Tatum,IND,33.5,PTS+AST,1266,2023-02-23
1,Jayson Tatum,IND,37.5,PTS+TRB,1266,2023-02-23
3,Jayson Tatum,IND,12.5,TRB+AST,1266,2023-02-23
4,Jayson Tatum,IND,0.5,BLK,1266,2023-02-23
5,Jayson Tatum,IND,2.5,TOV,1266,2023-02-23
...,...,...,...,...,...,...
278,Keegan Murray,POR,11.5,PTS,79897,2023-02-23
279,Kevin Huerter,POR,21.5,PTS+TRB+AST,87615,2023-02-23
280,Kevin Huerter,POR,17.5,PTS+AST,87615,2023-02-23
281,Kevin Huerter,POR,18.5,PTS+TRB,87615,2023-02-23


In [81]:
data.columns = ['Player', 'G', 'Date', 'Age', 'Tm', 'H/A', 'Opp', 'W/L', 'GS', 'MP', 'FG',
       'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB',
       'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc', '+/-']

In [82]:
data

,Player,G,Date,Age,Tm,H/A,Opp,W/L,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,Jayson Tatum,1,2022-10-18,24-229,BOS,NaN,PHI,W (+9),1,38.633,...,12,12,4,1,1,3,1,35,30.1,10.0
1,Jayson Tatum,2,2022-10-21,24-232,BOS,@,MIA,W (+7),1,36.783,...,3,5,4,1,2,2,1,29,22.7,-1.0
2,Jayson Tatum,3,2022-10-22,24-233,BOS,@,ORL,W (+6),1,37.650,...,8,8,1,1,2,1,2,40,34.2,6.0
3,Jayson Tatum,4,2022-10-24,24-235,BOS,@,CHI,L (-18),1,35.433,...,8,8,5,0,0,1,2,26,20.7,-8.0
4,Jayson Tatum,5,2022-10-28,24-239,BOS,NaN,CLE,L (-9),1,42.183,...,6,7,4,1,1,4,4,32,24.1,-10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18564,Michael Foster Jr.,1,2022-11-22,19-310,PHI,NaN,BRK,W (+9),0,1.033,...,0,0,0,0,0,0,0,0,0.0,-2.0
18565,Malcolm Hill,1,2022-10-24,26-363,CHI,NaN,BOS,W (+18),0,0.267,...,1,1,0,0,0,0,0,0,0.3,0.0
18566,Malcolm Hill,2,2022-10-26,27-000,CHI,NaN,IND,W (+15),0,0.650,...,0,0,0,0,0,0,0,0,0.0,-2.0
18567,Malcolm Hill,3,2022-11-07,27-012,CHI,NaN,TOR,W (+14),0,0.217,...,0,0,0,0,0,0,0,0,0.0,0.0


In [87]:
home = data['H/A'].to_list()

In [88]:
for i in range(len(home)):
    if home[i] == '@':
        home[i] = 0
    else:
        home[i] = 1
#home

[1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,


In [89]:
data['H/A'] = home
data

,Player,G,Date,Age,Tm,H/A,Opp,W/L,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,Jayson Tatum,1,2022-10-18,24-229,BOS,1,PHI,W (+9),1,38.633,...,12,12,4,1,1,3,1,35,30.1,10.0
1,Jayson Tatum,2,2022-10-21,24-232,BOS,0,MIA,W (+7),1,36.783,...,3,5,4,1,2,2,1,29,22.7,-1.0
2,Jayson Tatum,3,2022-10-22,24-233,BOS,0,ORL,W (+6),1,37.650,...,8,8,1,1,2,1,2,40,34.2,6.0
3,Jayson Tatum,4,2022-10-24,24-235,BOS,0,CHI,L (-18),1,35.433,...,8,8,5,0,0,1,2,26,20.7,-8.0
4,Jayson Tatum,5,2022-10-28,24-239,BOS,1,CLE,L (-9),1,42.183,...,6,7,4,1,1,4,4,32,24.1,-10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18564,Michael Foster Jr.,1,2022-11-22,19-310,PHI,1,BRK,W (+9),0,1.033,...,0,0,0,0,0,0,0,0,0.0,-2.0
18565,Malcolm Hill,1,2022-10-24,26-363,CHI,1,BOS,W (+18),0,0.267,...,1,1,0,0,0,0,0,0,0.3,0.0
18566,Malcolm Hill,2,2022-10-26,27-000,CHI,1,IND,W (+15),0,0.650,...,0,0,0,0,0,0,0,0,0.0,-2.0
18567,Malcolm Hill,3,2022-11-07,27-012,CHI,1,TOR,W (+14),0,0.217,...,0,0,0,0,0,0,0,0,0.0,0.0


In [55]:
type(data['MP'].iloc[0])

str

In [56]:
test = data['MP']

In [57]:
test

0        38:38
1        36:47
2        37:39
3        35:26
4        42:11
         ...  
18564     1:02
18565     0:16
18566     0:39
18567     0:13
18568     1:06
Name: MP, Length: 18569, dtype: object

In [74]:
test =  data['MP'].to_list()

for i in range(len(test)):
    new = test[i].split(':')
    res = float(new[0])+(float(new[1])/60)
    test[i] = res
data['MP'] = test

AttributeError: 'float' object has no attribute 'split'

In [75]:
data

,Player,G,Date,Age,Tm,,Opp,.1,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,Jayson Tatum,1,2022-10-18,24-229,BOS,NaN,PHI,W (+9),1,38.633,...,12,12,4,1,1,3,1,35,30.1,10.0
1,Jayson Tatum,2,2022-10-21,24-232,BOS,@,MIA,W (+7),1,36.783,...,3,5,4,1,2,2,1,29,22.7,-1.0
2,Jayson Tatum,3,2022-10-22,24-233,BOS,@,ORL,W (+6),1,37.650,...,8,8,1,1,2,1,2,40,34.2,6.0
3,Jayson Tatum,4,2022-10-24,24-235,BOS,@,CHI,L (-18),1,35.433,...,8,8,5,0,0,1,2,26,20.7,-8.0
4,Jayson Tatum,5,2022-10-28,24-239,BOS,NaN,CLE,L (-9),1,42.183,...,6,7,4,1,1,4,4,32,24.1,-10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18564,Michael Foster Jr.,1,2022-11-22,19-310,PHI,NaN,BRK,W (+9),0,1.033,...,0,0,0,0,0,0,0,0,0.0,-2.0
18565,Malcolm Hill,1,2022-10-24,26-363,CHI,NaN,BOS,W (+18),0,0.267,...,1,1,0,0,0,0,0,0,0.3,0.0
18566,Malcolm Hill,2,2022-10-26,27-000,CHI,NaN,IND,W (+15),0,0.650,...,0,0,0,0,0,0,0,0,0.0,-2.0
18567,Malcolm Hill,3,2022-11-07,27-012,CHI,NaN,TOR,W (+14),0,0.217,...,0,0,0,0,0,0,0,0,0.0,0.0


In [90]:
jay =data.loc[data['Player']== 'Jayson Tatum']
jay.describe()

,G,H/A,GS,MP,FG,FGA,FG%,3P,3PA,3P%,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
count,55.000,55.000,55.0,55.000,55.000,55.000,55.000,55.000,55.000,55.000,...,55.000,55.00,55.000,55.000,55.000,55.000,55.000,55.000,55.000,55.000
mean,28.000,0.527,1.0,37.292,9.945,21.418,0.461,3.327,9.309,0.343,...,7.564,8.60,4.545,1.036,0.782,2.891,2.182,30.636,23.153,7.255
std,16.021,0.504,0.0,4.117,3.021,3.619,0.113,1.806,2.508,0.141,...,2.923,3.37,2.124,0.981,0.896,1.812,1.172,8.294,8.273,13.843
min,1.000,0.000,1.0,27.417,3.000,15.000,0.188,0.000,4.000,0.000,...,1.000,1.00,1.000,0.000,0.000,0.000,0.000,12.000,2.900,-25.000
25%,14.500,0.000,1.0,35.633,8.000,19.000,0.383,2.000,7.000,0.273,...,6.000,7.00,3.000,0.000,0.000,2.000,1.000,26.000,18.000,1.000
50%,28.000,1.000,1.0,37.400,10.000,21.000,0.471,3.000,10.000,0.308,...,8.000,8.00,4.000,1.000,1.000,3.000,2.000,31.000,23.500,7.000
75%,41.500,1.000,1.0,39.525,12.000,24.000,0.525,4.000,10.500,0.442,...,9.000,11.00,6.000,1.500,1.000,4.000,3.000,35.000,28.700,14.000
max,55.000,1.000,1.0,48.117,15.000,29.000,0.667,8.000,16.000,0.667,...,17.000,19.00,10.000,4.000,4.000,9.000,4.000,51.000,42.700,46.000


In [91]:
jay.head()

,Player,G,Date,Age,Tm,H/A,Opp,W/L,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,Jayson Tatum,1,2022-10-18,24-229,BOS,1,PHI,W (+9),1,38.633,...,12,12,4,1,1,3,1,35,30.1,10.0
1,Jayson Tatum,2,2022-10-21,24-232,BOS,0,MIA,W (+7),1,36.783,...,3,5,4,1,2,2,1,29,22.7,-1.0
2,Jayson Tatum,3,2022-10-22,24-233,BOS,0,ORL,W (+6),1,37.650,...,8,8,1,1,2,1,2,40,34.2,6.0
3,Jayson Tatum,4,2022-10-24,24-235,BOS,0,CHI,L (-18),1,35.433,...,8,8,5,0,0,1,2,26,20.7,-8.0
4,Jayson Tatum,5,2022-10-28,24-239,BOS,1,CLE,L (-9),1,42.183,...,6,7,4,1,1,4,4,32,24.1,-10.0


In [92]:
jay.columns

Index(['Player', 'G', 'Date', 'Age', 'Tm', 'H/A', 'Opp', 'W/L', 'GS', 'MP',
       'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc', '+/-'],
      dtype='object')

In [93]:
jay.tail(5)

,Player,G,Date,Age,Tm,H/A,Opp,W/L,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
50,Jayson Tatum,51,2023-02-06,24-340,BOS,0,DET,W (+12),1,39.350,...,9,11,6,0,1,3,4,34,25.6,16.0
51,Jayson Tatum,52,2023-02-08,24-342,BOS,1,PHI,W (+7),1,37.400,...,7,8,9,1,1,4,2,12,8.7,5.0
52,Jayson Tatum,53,2023-02-10,24-344,BOS,1,CHO,W (+11),1,36.917,...,4,4,4,1,2,4,2,41,32.3,2.0
53,Jayson Tatum,54,2023-02-12,24-346,BOS,1,MEM,W (+10),1,37.983,...,7,7,3,2,2,3,0,16,9.4,7.0
54,Jayson Tatum,55,2023-02-15,24-349,BOS,1,DET,W (+18),1,34.033,...,9,9,7,0,0,4,3,38,29.6,1.0


In [94]:
jay = jay.loc['Date'< '2023-02-12']
jay

KeyError: 'False: boolean label can not be used without a boolean index'

In [98]:
x =jay['Date'][0]

In [99]:
x

Timestamp('2022-10-18 00:00:00')

In [107]:
mask = (jay['Date'] <= '2023-02-12')
df2 = jay.loc[mask]

In [109]:
df2.tail()

,Player,G,Date,Age,Tm,H/A,Opp,W/L,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
53,Jayson Tatum,54,2023-02-12,24-346,BOS,1,MEM,W (+10),1,37.983,...,7,7,3,2,2,3,0,16,9.4,7.0
